In [ ]:
import requests
import bcrypt
import re

# User Registration Function
def register_user():
    print("=== User Registration ===")
    name = input("Enter your name: ")

    # Validate email format
    email = input("Enter your email address: ")
    if not re.fullmatch(r'^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$', email):
        print("Invalid email format. Please try again.")
        return

    # Password validation
    password = input("Enter your password: ").encode('utf-8')
    if len(password) < 8:
        print("Password should be at least 8 characters long.")
        return
    if not re.fullmatch(r'^(?=.*[a-z])(?=.*[A-Z])(?=.*\d)(?=.*[-+_!@#$%^&*.,?]).+$', password.decode('utf-8')):
        print("Password must include uppercase, lowercase, number, and special character.")
        return

    # Hash the password and save to file
    hashed_password = bcrypt.hashpw(password, bcrypt.gensalt())
    security_answer = input("What is your pet's name? ").strip()

    with open("login.csv", 'a') as file:
        file.write(f"{name},{email},{hashed_password.decode('utf-8')},{security_answer}\n")
    print("Registration Successful!")

# Get Location using IP information
def get_user_location():
    try:
        response = requests.get("https://ipinfo.io/json?token=19bbba750f2b4f")
        if response.status_code == 200:
            latitude, longitude = response.json()['loc'].split(',')
            print(f"Detected location: Latitude {latitude}, Longitude {longitude}")
            return latitude, longitude
        else:
            print("Failed to detect location.")
            return None, None
    except:
        print("Error occurred while retrieving location.")
        return None, None

# Fetch Sunrise and Sunset Data
def fetch_sunrise_sunset_data():
    choice = input("Enter 1 for manual location or 2 for auto-detect: ")

    if choice == '1':
        latitude = input("Enter latitude: ")
        longitude = input("Enter longitude: ")
    elif choice == '2':
        latitude, longitude = get_user_location()
        if not latitude or not longitude:
            return
    else:
        print("Invalid choice.")
        return

    try:
        response = requests.get(f"https://api.sunrise-sunset.org/json?lat={latitude}&lng={longitude}&formatted=0")
        if response.status_code == 200:
            data = response.json()['results']
            print(f"Sunrise: {data['sunrise']}\nSunset: {data['sunset']}")
        else:
            print("Failed to retrieve data.")
    except:
        print("Error fetching sunrise and sunset data.")

# User Login Function
def user_login():
    print("=== User Login ===")
    attempts = 5

    while attempts > 0:
        email = input("Enter email: ")
        password = input("Enter password: ").encode('utf-8')

        with open("login.csv", 'r') as file:
            for line in file:
                name, stored_email, stored_password, _ = line.strip().split(",")
                if email == stored_email and bcrypt.checkpw(password, stored_password.encode('utf-8')):
                    print(f"Welcome, {name}!")
                    fetch_sunrise_sunset_data()
                    return
        attempts -= 1
        print(f"Invalid credentials. Attempts left: {attempts}")

    print("Too many failed attempts.")

# Forgot Password Function
def reset_password():
    print("=== Password Reset ===")
    email = input("Enter email: ")

    with open("login.csv", 'r') as file:
        records = [line.strip().split(",") for line in file]
        for record in records:
            if email == record[1]:
                answer = input("What is your pet's name? ")
                if answer.strip().lower() == record[3].strip().lower():
                    new_password = input("Enter new password: ").encode('utf-8')
                    if len(new_password) >= 8:
                        record[2] = bcrypt.hashpw(new_password, bcrypt.gensalt()).decode('utf-8')
                        with open("login.csv", 'w') as file:
                            file.writelines([','.join(r) + "\n" for r in records])
                        print("Password updated successfully!")
                        return
                    else:
                        print("Password must be at least 8 characters long.")
                else:
                    print("Incorrect answer.")
                return
        print("Email not found.")

# Main Program Loop
while True:
    print("\n1. Register\n2. Login\n3. Forgot Password\n4. Exit")
    choice = input("Enter choice: ")

    if choice == '1':
        register_user()
    elif choice == '2':
        user_login()
    elif choice == '3':
        reset_password()
    elif choice == '4':
        print("Goodbye!")
        break
    else:
        print("Invalid choice.")



1. Register
2. Login
3. Forgot Password
4. Exit
